In [1]:
import numpy as np
import healpy as hp
import glob
import matplotlib.pyplot as plt

from enterprise.pulsar import Pulsar
import skyplot_tools as spt

%matplotlib inline

In [2]:
golden_ratio = (np.sqrt(5.0)-1.0)/2.0

In [ ]:
def get_figsize(scale):
    fig_width_pt = 513.17 #469.755    # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0 / 72.27         # Convert pt to inch
    fig_width = fig_width_pt * inches_per_pt * scale  # width in inches
    fig_height = fig_width * golden_ratio             # height in inches
    fig_size = [fig_width, fig_height]
    return fig_size

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
          'figure.figsize': get_figsize(0.5),
          'axes.labelsize': 10,
          'lines.markersize': 4,
          'font.size': 10,
          'xtick.major.size':6,
          'xtick.minor.size':3,
          'ytick.major.size':6,
          'ytick.minor.size':3,
          'xtick.major.width':0.5,
          'ytick.major.width':0.5,
          'xtick.minor.width':0.5,
          'ytick.minor.width':0.5,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': 7,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'savefig.dpi':200,
          'path.simplify':True,
          #'font.family': 'serif',
          #'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          'text.usetex':True}

plt.rcParams.update(params)

In [4]:
#The pulsars we'll be analyzing
psrlist = ['J1713+0747',
           'J1909-3744',
           'J1640+2224',
           'J1600-3053',
           'J2317+1439',
           'J1918-0642',
           'J1614-2230',
           'J1744-1134',
           'J0030+0451',
           'J2145-0750',
           'J1857+0943',
           'J1853+1303',
           'J0613-0200',
           'J1455-3330',
           'J1741+1351',
           'J2010-1323',
           'J1024-0719',
           'J1012+5307',
#           'J0437-4715',
#           'J0034-0534',
          ]

In [4]:
datadir = '/home/pbaker/nanograv/data/nano11'
ephem = 'DE436'

with open(datadir + '/11yr_34.txt', 'r') as f:
    psrlist = [line.strip() for line in f]

# get PSR locations

ONLY NEED TO DO ONCE PER psrlist

In [5]:
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('_')[0] in psrlist]
timfiles = [x for x in timfiles if x.split('/')[-1].split('_')[0] in psrlist]

psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem=ephem)
    psrs.append(psr)

In [6]:
with open('11yr_locs.txt', 'w') as f:
    for p in psrs:
        phi = p.phi
        theta = np.pi/2 - p.theta
        f.write('{:s} {:f} {:f}\n'.format(p.name, phi, theta))

# Plot

In [7]:
psr_dict = {}
with open('11yr_locs.txt', 'r') as f:
    for line in f:
        line = line.split()
        psr_dict[line[0]] = [float(line[1]), float(line[2])]
    
psr_locs = []
for p in psrlist:
    psr_locs.append(psr_dict[p])
psr_locs = np.array(psr_locs)

In [12]:
nside = 8 #corresponds to 768 pixels
npix = hp.nside2npix(nside)

In [14]:
rundir = '/home/pbaker/nanograv/bwm/fixsky/latetime_BE/'
run, skymap, Ns, cors = np.loadtxt(rundir+'ULs.dat').T
skymap_smooth = hp.smoothing(skymap, 0.5)

Sigma is 729.938319 arcmin (0.212330 rad) 
-> fwhm is 1718.873385 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


In [15]:
ibest = np.argmin(skymap)
best = skymap[ibest]

print("min(N) = {:f}, best UL = {:.3e}".format(min(Ns), 10**best))

best_theta, best_phi = hp.pix2ang(nside,ibest)
best_theta = np.pi/2 - best_theta

min(N) = 52785.000000, best UL = 1.092e-14


In [16]:
print(best_theta, np.sin(best_theta), best_phi)

-0.08343008661061502 -0.08333333333333334 4.516039439535327


In [25]:
count = sum(skymap < np.log10(2.4e-14))
print(count, count/768)

325 0.4231770833333333


In [ ]:
ibest

In [ ]:
iworst = np.argmax(skymap)

w_theta, w_phi = hp.pix2ang(nside, iworst)
w_theta = np.pi/2 - w_theta
iworst

In [ ]:
print(np.min(skymap), np.max(skymap))

In [ ]:
#plt.rcParams.update(plt.rcParamsDefault)

sz = 11  # 7

plt.close('all')
plt.figure(0, figsize=(sz, sz*golden_ratio))
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap, cmap='viridis_r', vmin=-14, vmax=-12.95)

ax.scatter(psr_locs[:,0], psr_locs[:,1],
           s=256, marker='*', c='w', edgecolors='k', alpha=0.9)
#           marker='*', color='w', markersize=12, mew=1.0, mec='k', alpha=0.9)
#ax.plot(best_phi, best_theta,
#        '.', color='r', markersize=10, mew=1.0, mec='k', alpha=0.9)

#ax.set_title(r'11yr BWM ULs')

cbar = plt.colorbar(orientation='horizontal', pad=0.05)
plt.suptitle(r'$95\%$ Characteristic Strain Upper Limit, $h_{95}$', y=0.0, fontsize=20)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
#plt.savefig('/home/pbaker/skymap1.pdf')
plt.show()

In [ ]:
from matplotlib.colors import LogNorm
zmin = 1e-14
zmax = 1.2e-13
skymap_exp = 10**skymap
norm=LogNorm(vmin=zmin, vmax=zmax)

plt.close('all')
plt.figure(0, figsize=(7, 5))#7*golden_ratio))
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap_exp, cmap='viridis_r', norm=LogNorm(vmin=zmin, vmax=zmax))

ax.plot(psr_locs[:,0], psr_locs[:,1],
        '*', color='w', markersize=12, mew=1.0, mec='k', alpha=0.9)
#ax.plot(best_phi, best_theta,
#        '.', color='r', markersize=10, mew=1.0, mec='k', alpha=0.9)

#ax.set_title(r'11yr BWM ULs')

cbar = plt.colorbar(orientation='horizontal', pad=0.05)
plt.suptitle(r'$95\%$ Characteristic Strain Upper Limit, $h_{95}$', y=0.05)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('/home/pbaker/skymap2.pdf')
plt.show()

In [ ]:
plt.close('all')
plt.figure(1)
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap_smooth)

ax.plot(psr_locs[:,0], psr_locs[:,1], '*', color='w', markersize=10, mew=1.0, mec='k', alpha=0.9)    
ax.set_title(r'11yr BWM UL')

plt.colorbar(orientation='horizontal')
plt.suptitle(r'95\% Characteristic Strain Upper Limit, $A_{95}$', y=0.0)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('/home/pbaker/skymap_smooth.pdf')

In [ ]:
plt.close('all')
plt.figure(0)
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap)

ax.plot(psr_locs[:,0], psr_locs[:,1], '*', color='w', markersize=10, mew=1.0, mec='k', alpha=0.9)
ax.plot(best_phi, best_theta, '.', color='r', markersize=10, mew=1.0, mec='k', alpha=0.9)
ax.set_title(r'11yr BWM ULs')

ax.annotate('J1744-1134', xy=(4.7, -0.22), xytext=(6, -0.6), backgroundcolor='w',
            arrowprops=dict(color='w', shrink=0.02, width=2),
            )

plt.colorbar(orientation='horizontal')
plt.suptitle(r'95\% Characteristic Strain Upper Limit, $A_{95}$', y=0.0)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('/home/pbaker/skymap_arrow.pdf')
plt.show()

In [ ]:
import astropy
from astropy.time import Time

In [ ]:
ts = [56000, 57000]

a=Time(ts,format='mjd')

In [ ]:
a.decimalyear